# ACCESS Evaluation

## Functions

In [92]:
import pickle
import scipy.io as sio
from scipy.special import comb
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
from numpy import loadtxt
from numpy.random import randint as usample
from numpy.random import choice as samplenr
from numpy.random import multinomial
from numpy.random import normal
from numpy.random import binomial

In [112]:
# Utilities
def add_noise(S, p):
    n = S.shape[0]
    S_N = np.zeros((n, n))

    for i in range(n):
        for j in range(i+1, n):
            flip = binomial(1, p)
            if (S[i, j]==1):
                if (flip==1):
                    S_N[i, j] = 0
            else:
                if (flip==1):
                    S_N[i, j] = 1
    return S_N                

def compute_cost(S, Clustering):
    cost = 0
    n = S.shape[0]
    for i in range(n):
        for j in range(i+1, n):
            if ((Clustering[i]==Clustering[j]) and (S[i, j]==0)):
                cost += 1
            elif ((Clustering[i]!=Clustering[j]) and (S[i, j]==1)):
                cost += 1
    return cost

def find_cluster(v, P, S, Q):
    n = len(P)
    i = 0
    add_q = Q
    while (i<n):
        add_q += 1
        if (S[P[i], v]==1):
            return P[i], add_q
        else:
            i += 1
    return -1, add_q

# Algorithms
def KC(S):
    # Check input
    V = list(range(S.shape[0]))
    if (len(V)==0):
        return {}
    
    Clustering = {}
    Q = 0
    k = 1
    while V:
        # Choose pivot
        piv = V[usample(len(V))]
        Clustering[piv], V_left = k, []
        V.remove(piv)
        
        # Build pivot's cluster
        for i in V:
            Q += 1
            if (S[piv, i]==1):
                Clustering[i] = k
            else:
                V_left.append(i)
        k += 1
        V = V_left
        
    return Clustering, Q

def ACCESS_sqrt_wor(S, B, alpha):
    # Check input
    V = list(range(S.shape[0]))
    if (len(V)==0):
        return {}
    
    Clustering = {}
    Q = 0
    k = 1
    phi = 0
    while ((phi<B) and (len(V)>1)):
        # Choose pivot
        piv = V[usample(len(V))]
        Clustering[piv], V_left = k, []
        V.remove(piv)
        
        #Build pivot's cluster
        i = 0
        f = int(len(V)**alpha)
        neigh = np.random.choice(len(V), f, replace=False)
        while (i<f):
            if (S[piv, V[neigh[i]]]==1):
                Q -= (i+1)
                for j in V:
                    Q += 1
                    if (S[piv, j]==1):
                        Clustering[j] = k
                    else:
                        V_left.append(j)
                break
            else:
                Q += 1
                i += 1
        k += 1
        if (i==f):
            V_left = V
        V = V_left
        phi += 1
        
    for i in V:
        Clustering[i] = k
        k = +1
            
    return Clustering, Q

# Script for running experiments varing the parameter f(n)
def ex_1(S, rep, Q_value, p):
    Delta_KC, Delta_ACC = np.zeros(rep), np.zeros((Q_value, rep))
    Q_KC, Q_ACC = np.zeros(rep), np.zeros((Q_value, rep))
    
    n = int(S.shape[0])
    alpha = np.linspace(0, p, Q_value)
    B = n**alpha
    
    # KC
    for i in range(rep):
        Clustering_KC, Q_KC[i] = KC(S)
        Delta_KC[i] = compute_cost(S, Clustering_KC)

    Delta_BL = np.mean(Delta_KC) * np.ones(Q_value)
    Q_BL = np.mean(Q_KC) *np.ones(Q_value)

    # ACCESS
    for q in range(0, Q_value):
        for i in range(rep):
            Clustering_ACC, Q_ACC[q, i] = ACCESS_sqrt_wor(S, int(B[q]), alpha[q])
            Delta_ACC[q, i] = compute_cost(S, Clustering_ACC)

    Delta, Q = np.mean(Delta_ACC, axis=1), np.mean(Q_ACC, axis=1)
    Delta_std, Q_std = np.std(Delta_ACC, axis=1), np.std(Q_ACC, axis=1)
    
    return B, np.column_stack((Delta_BL, Q_BL)), np.column_stack((Delta, Delta_std)), np.column_stack((Q, Q_std))

## Data import

The code below creates similarity matrices with OPT = 0.

In [109]:
# Build the similarity matrices
# Skew
gold_skew = loadtxt('Data/skew/gold.txt').astype(int)
n = int(gold_skew.shape[0])
S_skew = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if (gold_skew[i, 1]==gold_skew[j, 1]):
            S_skew[i, j] = 1
        else:
            S_skew[i, j] = 0
            
# Sqrt
gold_sqrt = loadtxt('Data/sqrt/gold.txt').astype(int)
n = int(gold_sqrt.shape[0])
S_sqrt = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if (gold_sqrt[i, 1]==gold_sqrt[j, 1]):
            S_sqrt[i, j] = 1
        else:
            S_sqrt[i, j] = 0
            
# Landmarks
gold_landmarks = loadtxt('Data/landmarks/gold.txt').astype(int)
n = int(gold_landmarks.shape[0])
S_landmarks = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if (gold_landmarks[i, 1]==gold_landmarks[j, 1]):
            S_landmarks[i, j] = 1
        else:
            S_landmarks[i, j] = 0
            
# Captchas
gold_captchas = loadtxt('Data/captchas/gold.txt').astype(int)
n = int(gold_captchas.shape[0])
S_captchas = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if (gold_captchas[i, 1]==gold_captchas[j, 1]):
            S_captchas[i, j] = 1
        else:
            S_captchas[i, j] = 0

# Gym
gold_gym = loadtxt('Data/gym/gold.txt').astype(int)
n = int(gold_gym.shape[0])
S_gym = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if (gold_gym[i, 1]==gold_gym[j, 1]):
            S_gym[i, j] = 1
        else:
            S_gym[i, j] = 0

# Cora
gold_cora = loadtxt('Data/cora/gold.txt').astype(int)
n = int(gold_cora.shape[0])
S_cora = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if (gold_cora[i, 1]==gold_cora[j, 1]):
            S_cora[i, j] = 1
        else:
            S_cora[i, j] = 0

## Experiments: Q vs Delta

## OPT = 0

Requires:
    1) the similiraty matrix; 
    2) the number of repetion; 
    3) the number of values for f(n) to test;
    4) the parameter p which controls the range for f(n) via $f(n) \in [n^0, n^p]$.
    
Plot $Q$ vs $\Delta$

In [110]:
# Parameters
rep = 50
Q_value = 20
p = 3/4

B, BL_skew, Delta_skew, Q_skew = ex_1(S_skew, rep, Q_value, p)
B, BL_sqrt, Delta_sqrt, Q_sqrt = ex_1(S_sqrt, rep, Q_value, p)
B, BL_landmarks, Delta_landmarks, Q_landmarks = ex_1(S_landmarks, rep, Q_value, p)
B, BL_captchas, Delta_captchas, Q_captchas = ex_1(S_captchas, rep, Q_value, p)
B, BL_gym, Delta_gym, Q_gym = ex_1(S_gym, rep, Q_value, p)
B, BL_cora, Delta_cora, Q_cora = ex_1(S_cora, rep, Q_value, p)

np.savez('skew', B, BL_skew, Delta_skew, Q_skew)
np.savez('sqrt', B, BL_sqrt, Delta_sqrt, Q_sqrt)
np.savez('landmarks', B, BL_landmarks, Delta_landmarks, Q_landmarks)
np.savez('captchas', B, BL_captchas, Delta_captchas, Q_captchas)
np.savez('gym', B, BL_gym, Delta_gym, Q_gym)
np.savez('cora', B, BL_cora, Delta_cora, Q_cora)

## OPT > 0

In [113]:
noise = 0.2 #noise rate

# Generate a noisy version of the given similarity matrix
Sn_skew = add_noise(S_skew, noise)
Sn_sqrt = add_noise(S_sqrt, noise)
Sn_landmarks = add_noise(S_landmarks, noise)
Sn_captchas = add_noise(S_captchas, noise)
Sn_gym = add_noise(S_gym, noise)
Sn_cora = add_noise(S_cora, noise)

B, BL_skewn, Delta_skewn, Q_skewn = ex_1(Sn_skew, rep, Q_value, p)
B, BL_sqrtn, Delta_sqrtn, Q_sqrtn = ex_1(Sn_sqrt, rep, Q_value, p)
B, BL_landmarksn, Delta_landmarksn, Q_landmarksn = ex_1(Sn_landmarks, rep, Q_value, p)
B, BL_captchasn, Delta_captchasn, Q_captchasn = ex_1(Sn_captchas, rep, Q_value, p)
B, BL_gymn, Delta_gymn, Q_gymn = ex_1(Sn_gym, rep, Q_value, p)
B, BL_cora, Delta_cora, Q_cora = ex_1(Sn_cora, rep, Q_value, p)

np.savez('skew_two', B, BL_skewn, Delta_skewn, Q_skewn)
np.savez('sqrt_two', B, BL_sqrtn, Delta_sqrtn, Q_sqrtn)
np.savez('landmarks_two', B, BL_landmarksn, Delta_landmarksn, Q_landmarksn)
np.savez('captchas_two', B, BL_captchasn, Delta_captchasn, Q_captchasn)
np.savez('gym_two', B, BL_gymn, Delta_gymn, Q_gymn)
np.savez('cora_two', B, BL_cora, Delta_cora, Q_cora)

noise = 0.1 #noise rate

# Generate a noisy version of the given similarity matrix
Sn_skew = add_noise(S_skew, noise)
Sn_sqrt = add_noise(S_sqrt, noise)
Sn_landmarks = add_noise(S_landmarks, noise)
Sn_captchas = add_noise(S_captchas, noise)
Sn_gym = add_noise(S_gym, noise)
Sn_cora = add_noise(S_cora, noise)

B, BL_skewn, Delta_skewn, Q_skewn = ex_1(Sn_skew, rep, Q_value, p)
B, BL_sqrtn, Delta_sqrtn, Q_sqrtn = ex_1(Sn_sqrt, rep, Q_value, p)
B, BL_landmarksn, Delta_landmarksn, Q_landmarksn = ex_1(Sn_landmarks, rep, Q_value, p)
B, BL_captchasn, Delta_captchasn, Q_captchasn = ex_1(Sn_captchas, rep, Q_value, p)
B, BL_gymn, Delta_gymn, Q_gymn = ex_1(Sn_gym, rep, Q_value, p)
B, BL_cora, Delta_cora, Q_cora = ex_1(Sn_cora, rep, Q_value, p)

np.savez('skew_one', B, BL_skewn, Delta_skewn, Q_skewn)
np.savez('sqrt_one', B, BL_sqrtn, Delta_sqrtn, Q_sqrtn)
np.savez('landmarks_one', B, BL_landmarksn, Delta_landmarksn, Q_landmarksn)
np.savez('captchas_one', B, BL_captchasn, Delta_captchasn, Q_captchasn)
np.savez('gym_one', B, BL_gymn, Delta_gymn, Q_gymn)
np.savez('cora_one', B, BL_cora, Delta_cora, Q_cora)

noise = 0.3 #noise rate

# Generate a noisy version of the given similarity matrix
Sn_skew = add_noise(S_skew, noise)
Sn_sqrt = add_noise(S_sqrt, noise)
Sn_landmarks = add_noise(S_landmarks, noise)
Sn_captchas = add_noise(S_captchas, noise)
Sn_gym = add_noise(S_gym, noise)
Sn_cora = add_noise(S_cora, noise)

B, BL_skewn, Delta_skewn, Q_skewn = ex_1(Sn_skew, rep, Q_value, p)
B, BL_sqrtn, Delta_sqrtn, Q_sqrtn = ex_1(Sn_sqrt, rep, Q_value, p)
B, BL_landmarksn, Delta_landmarksn, Q_landmarksn = ex_1(Sn_landmarks, rep, Q_value, p)
B, BL_captchasn, Delta_captchasn, Q_captchasn = ex_1(Sn_captchas, rep, Q_value, p)
B, BL_gymn, Delta_gymn, Q_gymn = ex_1(Sn_gym, rep, Q_value, p)
B, BL_cora, Delta_cora, Q_cora = ex_1(Sn_cora, rep, Q_value, p)

np.savez('skew_three', B, BL_skewn, Delta_skewn, Q_skewn)
np.savez('sqrt_three', B, BL_sqrtn, Delta_sqrtn, Q_sqrtn)
np.savez('landmarks_three', B, BL_landmarksn, Delta_landmarksn, Q_landmarksn)
np.savez('captchas_three', B, BL_captchasn, Delta_captchasn, Q_captchasn)
np.savez('gym_three', B, BL_gymn, Delta_gymn, Q_gymn)
np.savez('cora_three', B, BL_cora, Delta_cora, Q_cora)

q =  0
q =  1
q =  2
q =  3
q =  4
q =  5
q =  6
q =  7
q =  8
q =  9
q =  10
q =  11
q =  12
q =  13
q =  14
q =  15
q =  16
q =  17
q =  18
q =  19
q =  0
q =  1
q =  2
q =  3
q =  4
q =  5
q =  6
q =  7
q =  8
q =  9
q =  10
q =  11
q =  12
q =  13
q =  14
q =  15
q =  16
q =  17
q =  18
q =  19
q =  0
q =  1
q =  2
q =  3
q =  4
q =  5
q =  6
q =  7
q =  8
q =  9
q =  10
q =  11
q =  12
q =  13
q =  14
q =  15
q =  16
q =  17
q =  18
q =  19
